In [1]:
from collections import deque
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

from tqdm.notebook import trange

In [2]:
class Resnet(nn.Module):

    def __init__(self, in_channels, out_channels, resnet_blocks = 5):
        super(Resnet, self).__init__()
        self.start = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size = 3, padding = 1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU()
        )

        self.blocks = nn.ModuleList(
            [ResidualConnection(out_channels) for _ in range(resnet_blocks)]
        )

        self.policy_head = nn.Sequential(
            nn.Conv2d(out_channels, out_channels = 32, kernel_size = 3, padding = 1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Flatten(),
            nn.Linear(32 * 6 * 7, 7)
        )

        self.value_head = nn.Sequential(
            nn.Conv2d(out_channels, out_channels = 3, kernel_size = 3, padding = 1),
            nn.BatchNorm2d(3),
            nn.ReLU(),
            nn.Flatten(),
            nn.Linear(3 * 6 * 7, 256),
            nn.ReLU(),
            nn.Linear(256, 1),
            nn.Tanh()
        )

    def forward(self, x):
        x = self.start(x)
        for block in self.blocks:
            x = block(x)
        policy = self.policy_head(x)
        value = self.value_head(x)
        return policy, value
        

class ResidualConnection(nn.Module):

    def __init__(self, out_channels):
        super(ResidualConnection, self).__init__()

        self.block = nn.Sequential(
            nn.Conv2d(in_channels = out_channels, out_channels = out_channels, kernel_size = 3, padding = 1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(),
            nn.Conv2d(in_channels = out_channels, out_channels = out_channels, kernel_size = 3, padding = 1),
            nn.BatchNorm2d(out_channels)     
        )

    def forward(self, x):
        features = self.block(x)
        return torch.relu(features + x)

In [3]:
class Connect4:

    def __init__(self):
        self.W = 7
        self.H = 6
        self.board = np.zeros((self.H, self.W))
        self.all_moves = np.arange(self.W)


    def reset(self):
        self.board = np.zeros((self.H, self.W))
        return self.board
        
    def get_valid_moves(self, state):
        return (state[0] == 0).astype(int)

    def step(self, board, move, player):
        board = board.copy()
        row, col = move//self.W, move%self.W
        brow, bcol = self.H-1, col
        while True:
            if board[brow, bcol] == 0:
                board[brow, bcol] = player
                break
            brow -= 1
        done, result = self.is_terminal(board)
        return board, result, done

    def is_terminal(self, board,H = 6, W = 7):
        for r in range(H):
            for c in range(W - 3):
                val = board[r, c]
                if val in [1, -1] and val == board[r, c+1] == board[r, c+2] == board[r, c+3]:
                    return True, val
        for r in range(H - 3):
            for c in range(W):
                val = board[r, c]
                if val in [1, -1] and val == board[r+1, c] == board[r+2, c] == board[r+3, c]:
                    return True, val
        for r in range(H - 3):
            for c in range(W - 3):
                val = board[r, c]
                if val in [1, -1] and val == board[r+1, c+1] == board[r+2, c+2] == board[r+3, c+3]:
                    return True, val
        for r in range(3, H):
            for c in range(W - 3):
                val = board[r, c]
                if val in [1, -1] and val == board[r-1, c+1] == board[r-2, c+2] == board[r-3, c+3]:
                    return True, val
        if not (board == 0).any():
            return True, 0
        return False, 0
        
    def stackedStates(self, state, current_player):
        
        return np.stack((
            state == current_player, 
            state == -current_player,
            state == 0
        )).astype(np.float32)

    
    def show(self, state):
        print(state)

In [4]:
import numpy as np

class Node:

    def __init__(self, state, parent, move, player, prob = 0):
        self.H = 6
        self.W = 7
        self.state = state
        self.parent = parent
        self.move = move

        self.player = player
        self.children = {}
        
        self.prob = prob
        
        self.N = 0
        self.W = 0
        
    def is_fully_expanded(self):
        return len(np.argwhere(self.state[0] == 0)) == 0

    
    def UCB1(self, c = 2):
        if self.N == 0:
            return float('inf')
        return self.W/self.N + c * self.prob * np.sqrt(self.parent.N)/(self.N + 1)

In [5]:
class MCTS:

    def __init__(self, env, root_state, root_player, model, device, PARAMS):
        self.env = env
        self.H = 6
        self.W = 7
        self.policy = 0
        self.model = model
        self.device = device
        self.root = Node(state = root_state,
                         parent = None,
                         move = None,
                         player = root_player
                        )
        self.PARAMS = PARAMS
        
    def selection(self):
        current = self.root
        
        while True:
            is_terminal, _ = self.env.is_terminal(current.state)
            if is_terminal:
                return current
            if current.is_fully_expanded():
                return current
            if len(current.children) == 0:
                return current
            best_child = max(current.children.values(), key = lambda c: c.UCB1())
            current = best_child
            
        return current

    def expansion(self, node, policy):
        valid_states = self.env.get_valid_moves(node.state)

        for action, prob in enumerate(policy):
            if valid_states[action] == 1 and action not in node.children:
                new_state = self.env.step(node.state, action, node.player)[0]
                child = Node(state = new_state,
                             parent = node,
                             move = action,
                             player = -node.player,
                             prob = prob
                            )
                node.children[action] = child
                
    def backpropagation(self, child, value):
        current = child
        while current is not None:
            current.N += 1
            current.W += value
            value = -value
            current = current.parent

    @torch.no_grad()
    def search(self):
    
        for _ in range(self.PARAMS['SEARCHES']):
            node = self.selection()
            is_terminal, value = self.env.is_terminal(node.state)
            
            if is_terminal:
                value = value * node.player
            else:
                stackedStates = self.env.stackedStates(node.state, node.player)
                policy, value = self.model(torch.tensor(stackedStates, dtype = torch.float).unsqueeze(0).to(self.device))
                policy = torch.softmax(policy, dim = 1).squeeze(0).cpu().detach().numpy()
                if node is self.root:
                    alpha = self.PARAMS['ALPHA']
                    eps = self.PARAMS['EPSILON']
                    noise = np.random.dirichlet([alpha] * len(self.env.all_moves))
                    policy = (1 - eps) * policy + eps * noise

                value = value.squeeze(0).cpu().item()
                valid_moves = self.env.get_valid_moves(node.state)
                policy = policy * valid_moves
                if np.sum(policy)>0:
                    policy = policy/(np.sum(policy))
                else:
                    policy = valid_moves/np.sum(valid_moves)
    
                self.expansion(node, policy)
                if len(node.children) > 0:
                    best_action = max(node.children.keys(), key=lambda a: node.children[a].prob)
                    child = node.children[best_action]
                else:
                    child = node
            target_node = node if is_terminal else child
    
            self.backpropagation(target_node, value)
        actions = np.zeros(7)

        if len(self.root.children) == 0:
            valid_moves = self.env.get_valid_moves(self.root.state)
            action_probs = np.array(valid_moves / np.sum(valid_moves))
            return action_probs

        total_visits = sum(child.N for child in self.root.children.values())
        if total_visits > 0:
            for action, child in self.root.children.items():
                actions[action] = child.N
            actions /= total_visits
        else:
            valid_moves = self.env.get_valid_moves(self.root.state)
            actions = np.array(valid_moves / np.sum(valid_moves))
        return actions

    def __repr__(self):
        return f"{self.root.state}\n{self.root.parent}\n{self.root.children}\n{self.root.action}\n{self.root.children.n}"

In [6]:
class AlphaZero:

    def __init__(self, env, model, policy_loss, value_loss, optimizer, device, PARAMS, current_player = 1):
        self.env = env
        self.model = model
        self.model.to(device)
        self.optimizer = optimizer
        self.mainBuffer = deque(maxlen = 30000)
        self.policy_loss = policy_loss
        self.value_loss = value_loss
        self.device = device
        self.current_player = current_player
        self.tau = 1.25
        self.PARAMS = PARAMS

    
    def selfPlayData(self, ):
        buffer = []
        state = self.env.reset()
        current_player = 1
        
        while True:
            root = MCTS(self.env, state, current_player, self.model, self.device, self.PARAMS)

            actions = root.search()
            temperature_probs = actions ** (1/self.PARAMS['TAU'])
            temperature_probs /= np.sum(temperature_probs)
            buffer.append((state, actions, current_player))
            action = np.random.choice(self.env.all_moves, p = temperature_probs)
            state, result, done = self.env.step(state, action, current_player)
            if done:
                mainBuffer = []
                for states, actions, player in buffer:
                    if result == 0:
                        value = 0
                    elif result == player:
                        value = 1
                    else:
                        value = -1
                    mainBuffer.append((self.env.stackedStates(states, current_player), actions, value))
                return mainBuffer

            current_player = -current_player

    def train(self):
        for i in trange(self.PARAMS['TOTAL_ITERATIONS']):
            self.model.eval()
            self.mainBuffer = []
            for _ in trange(self.PARAMS['SELF_PLAY_ITERATIONS']):
                self.mainBuffer.extend(self.selfPlayData())
            
            self.model.train()
            losses = []
            epoch_loss = 0
            previous_model = self.model
            for epoch in trange(self.PARAMS['EPOCHS']):

                epoch_loss = 0
                batch_count = 0
                np.random.shuffle(self.mainBuffer)
                for idx in range(0, len(self.mainBuffer), self.PARAMS["BATCH_SIZE"]):
                    data = self.mainBuffer[idx:idx+self.PARAMS["BATCH_SIZE"]]
                    states, actions, value = zip(*data)
    
                    actions = torch.tensor(np.array(actions), dtype = torch.float).to(self.device)
                    values = torch.tensor(value, dtype = torch.float).unsqueeze(1).to(self.device)
                    states = torch.tensor(states, dtype = torch.float).to(self.device)

                    model_policy, model_value = self.model(states)

                    policy_loss = self.policy_loss(model_policy, actions)
                    value_loss = self.value_loss(model_value, values)
                    total_loss = policy_loss + value_loss
                    
                    self.optimizer.zero_grad()
                    total_loss.backward()
                    torch.nn.utils.clip_grad_norm_(self.model.parameters(), max_norm=1.0)
                    self.optimizer.step()            

                    epoch_loss += total_loss.item()
                    batch_count += 1

                avg_loss = epoch_loss/batch_count
                    
                print(f"Epoch {epoch + 1}\tTotal Loss : {avg_loss}")
                losses.append(avg_loss)
            
            self.evaluation(previous_model)
            self.save_model()

    def save_model(self):

        checkpoint = {
            "model" : self.model.state_dict(),
            "optimizer_state" : self.optimizer.state_dict()
        }
        torch.save(checkpoint, 'checkpoint.pth')
        print("Hit Checkpoint")
        
        
    

    def evaluation(self, pmodel):
        wins = 0
        self.model.eval()

        for game in trange(self.PARAMS['EVALUATION_GAMES']):
                state = self.env.reset()
                current_player = 1

                while True:
                    root = MCTS(self.env, state, current_player, self.model, self.device, self.PARAMS)
                    actions = root.search()
                    action = np.argmax(actions)
                    state, result, done = self.env.step(state, action, current_player)
                    if done:
                        if result == current_player:
                            wins += 1
                        break
                    current_player = -current_player

                    root = MCTS(self.env, state, current_player, pmodel, self.device, self.PARAMS)
                    actions = root.search()
                    action = np.argmax(actions)
                    state, result, done = self.env.step(state, action, current_player)
                    if done:
                        break
                    current_player = -current_player

        win_rate = wins/self.PARAMS["EVALUATION_GAMES"]
        print(f"Evaluation : {wins}/{self.PARAMS['EVALUATION_GAMES']} Wins : {wins}")
                            

In [24]:
PARAMETERS = {
    
    "IN_CHANNELS" : 3,
    "OUT_CHANNELS" : 64,
    "RESNET_BLOCKS" : 5,
    
    "SELF_PLAY_ITERATIONS" : 500,
    "EPOCHS" : 20,
    "BATCH_SIZE" : 128,
    "TOTAL_ITERATIONS" : 20,
    
    "SEARCHES" : 50,
    "EVALUATION_GAMES" : 20,
    
    "ALPHA" : 0.3,
    "EPSILON" : 0.25,
    "TAU" : 1.25
    }

In [8]:
env = Connect4()

model = Resnet(PARAMETERS['IN_CHANNELS'], 
               PARAMETERS['OUT_CHANNELS'], 
               PARAMETERS['RESNET_BLOCKS']
              )

optimizer = optim.Adam(model.parameters(), lr = 0.001, weight_decay = 1e-4)
policy_loss = nn.CrossEntropyLoss()
value_loss = nn.MSELoss()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

agent= AlphaZero(env, 
                 model, 
                 policy_loss, 
                 value_loss, 
                 optimizer, 
                 device, 
                 PARAMETERS
                )

agent.train()

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

/tmp/ipykernel_47/3078040116.py:67: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /pytorch/torch/csrc/utils/tensor_new.cpp:254.)
  states = torch.tensor(states, dtype = torch.float).to(self.device)


Epoch 1	Total Loss : 2.7432805454289473
Epoch 2	Total Loss : 2.3888552718692355
Epoch 3	Total Loss : 2.2750759809105485
Epoch 4	Total Loss : 2.196598357624478
Epoch 5	Total Loss : 2.1468663668190993
Epoch 6	Total Loss : 2.1211720020682723
Epoch 7	Total Loss : 2.0944228768348694
Epoch 8	Total Loss : 2.0807184599064015
Epoch 9	Total Loss : 2.050313030128126
Epoch 10	Total Loss : 2.053801982491105
Epoch 11	Total Loss : 2.034401912380148
Epoch 12	Total Loss : 2.0273858750308
Epoch 13	Total Loss : 2.0292916816693767
Epoch 14	Total Loss : 2.0161350943424083
Epoch 15	Total Loss : 2.0096707730381578
Epoch 16	Total Loss : 2.0008855561415353
Epoch 17	Total Loss : 1.9935759416332952
Epoch 18	Total Loss : 1.9892831919369873
Epoch 19	Total Loss : 1.9857241345776453
Epoch 20	Total Loss : 1.9824634326828852


...

  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

Epoch 1	Total Loss : 1.6315011860014081
Epoch 2	Total Loss : 1.4575936074729439
Epoch 3	Total Loss : 1.36762746712109
Epoch 4	Total Loss : 1.3070482445192766
Epoch 5	Total Loss : 1.2585936505515296
Epoch 6	Total Loss : 1.2230820387333363
Epoch 7	Total Loss : 1.197868503965773
Epoch 8	Total Loss : 1.1819635414862417
Epoch 9	Total Loss : 1.167873273024688
Epoch 10	Total Loss : 1.156662035632778
Epoch 11	Total Loss : 1.1518864556475803
Epoch 12	Total Loss : 1.1429979103105563
Epoch 13	Total Loss : 1.1385561850693848
Epoch 14	Total Loss : 1.1357672858882595
Epoch 15	Total Loss : 1.1280222422367818
Epoch 16	Total Loss : 1.1253701373263523
Epoch 17	Total Loss : 1.1214470723727803
Epoch 18	Total Loss : 1.1219836204975575
Epoch 19	Total Loss : 1.1201780959292575
Epoch 20	Total Loss : 1.127270497717299


  0%|          | 0/20 [00:00<?, ?it/s]

Evaluation : 16/20 Wins : 16
Hit Checkpoint


In [25]:
env = Connect4()

model = Resnet(PARAMETERS['IN_CHANNELS'], 
               PARAMETERS['OUT_CHANNELS'], 
               PARAMETERS['RESNET_BLOCKS']
              )


# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device= "cpu"
model_checkpoint = torch.load("/kaggle/input/checkpoint1/pytorch/default/1/checkpoint1.pth", map_location= "cpu")
model.load_state_dict(model_checkpoint["model"])

model.eval()

state = env.reset()
current_player = 1

while True:
    root = MCTS(env, state, current_player, model, device, PARAMETERS)
    actions = root.search()
    action = np.argmax(actions)
    state, result, done = env.step(state, action, current_player)
    env.show(state)
    if done:
        if result == current_player:
            wins += 1
            print("Bot Won!!")
        break
    current_player = -current_player

    print(f"Available Moves : {np.where(env.get_valid_moves(state)==1)}")
    action = int(input("Enter action (0-6) : "))
    state, result, done = env.step(state, action, current_player)
    if done:
        if result == current_player:
            print("You Won!!")
        break
    current_player = -current_player
    env.show(state)


[[0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0.]]
Available Moves : (array([0, 1, 2, 3, 4, 5, 6]),)


Enter action (0-6) :  6


[[ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.  0. -1.]]
[[ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.  0. -1.]]
Available Moves : (array([0, 1, 2, 3, 4, 5, 6]),)


Enter action (0-6) :  2


[[ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0. -1.  0.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.  0. -1.]]
[[ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0. -1.  0.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.  0.  0.]
 [ 0.  0.  1.  0.  1.  0. -1.]]
Available Moves : (array([0, 1, 2, 3, 4, 5, 6]),)


...

[[ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0. -1.  0.  0.  0.  0.]
 [ 1.  0.  1. -1.  0.  0.  0.]
 [ 1. -1.  1. -1.  1.  0. -1.]]
[[ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0. -1.  0.  0.  0.  0.]
 [ 1.  0.  1. -1.  0.  0.  1.]
 [ 1. -1.  1. -1.  1.  0. -1.]]
Available Moves : (array([0, 1, 2, 3, 4, 5, 6]),)


Enter action (0-6) :  3


[[ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0. -1. -1.  0.  0.  0.]
 [ 1.  0.  1. -1.  0.  0.  1.]
 [ 1. -1.  1. -1.  1.  0. -1.]]
[[ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0. -1. -1.  0.  0.  0.]
 [ 1.  1.  1. -1.  0.  0.  1.]
 [ 1. -1.  1. -1.  1.  0. -1.]]
Available Moves : (array([0, 1, 2, 3, 4, 5, 6]),)


Enter action (0-6) :  3


You Won!!
